In [2]:

# !pip install mysql-connector-python

# levantar la base de datos: mysql.server start
# bajar la base de datos: mysql.server stop
# reiniciar la base de datos: mysql.server restart

# ANTES DE INICIAR, SE CARGARON TODOS LOS EXCEL PASADOS A LA BASE DE DATOS.
# La data original fue cargada como tablas y desde SQL convertidos al nuevo
# modelo (tablas que inician con f_%)

# para exportar la base de datos
# mysqldump -u proyectodegrado2 -p proyectodegrado2 -h localhost > archivo.sql
# pide password: pg2

# para importar la base de datos
# mysql -u proyectodegrado2 -p proyectodegrado2 -h localhost < archivo.sql
# pide password: pg2
# la bsae de datos debe estar creada y el usuario uaopiloto2 tambien con password up2 y 
# permisos de sql todos, y de administrador le puse dba


import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
import mysql.connector   # es bueno instalar sqlalchemy y mysql.connector para facilitar luego DML y SELECT.

usuario = 'proyectodegrado2'
contrasena = 'pg2'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'proyectodegrado2'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contrasena}@{host}/{base_de_datos}'

# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)


Todas las transformaciones se hicieron en el archivo sql en **transformacion_datos.sql**


In [281]:
# ejemplo de cómo ejecutar un DML
drop_tabla = text(""" drop table temp""")

crea_tabla = text("""
    create table temp as select count(*) cuantos from f_estudiantes
""") 

# Ejecuta la instrucción SQL utilizando la conexión del motor
with motor.connect() as connection:
    try:
        connection.execute(drop_tabla)
        print('borró temp')
    except:
        pass # si la tabla no existe saca error porque no puede hacer el drop

    connection.execute(crea_tabla)
    print('creó temp')

borró temp
creó temp


## **0. Creamos la tabla base DATOS basada en F_MatriculasAsignaturas**

Para las filas con marca "analizar = 1", se indica cuales filas tienen que ver con matrículas de las asignaturas de interés.  en este caso:  300MAG006 y 300MAG007.

hay casos de filas sin docente, entonces, usamos el docente 'Ninguno', con edad promedio (43), y sexo mayority (M).

MEJORAR:
Se puede luego optimizar el cálculo de la edad promedio al momento de determinar cual asigantura está tomando el estudiante

In [282]:
# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT  ma.key_estudiante, ma.periodo, ma.asignatura, 
		TIMESTAMPDIFF(YEAR,e.birthdate,p.fecha_inicio) estudiante_edad, 
        e.sex estudiante_genero, IFNULL(e.cod_colegio,'Ninguno') colegio,
        IFNULL(d.id_docente,-1) id_docente, IFNULL(d.sex,1) docente_genero,
        IFNULL(TIMESTAMPDIFF(YEAR,d.birthdate,p.fecha_inicio),43) docente_edad, 
        ma.estado, IFNULL(CAST(ma.nota AS DECIMAL(10,2)),-1) nota
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_estudiantes e
    ON e.key_estudiante = ma.key_estudiante
    LEFT JOIN f_docentes d 
    ON d.key_docente = ma.key_docente
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    WHERE ma.analizar = 1;
"""

DATOS = pd.read_sql(query, con=motor)
DATOS.head()


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,nota
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,3.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,1.5
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,2.2
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,1.7
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,1.8


In [283]:
DATOS.shape

(7145, 11)

pivoteaemos primero el **docente** con que tomó la clase para hacer hot_encoding

In [284]:
temp = DATOS[['key_estudiante','periodo','asignatura','id_docente']].copy()
temp['valor'] = 1
temp.head()

,key_estudiante,periodo,asignatura,id_docente,valor
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,-1,1
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,-1,1
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,-1,1
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,-1,1
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,-1,1


In [285]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='id_docente', values='valor').reset_index()
temp_pivot = temp_pivot.add_prefix('docente_')
temp_pivot.rename(columns={'docente_key_estudiante':'key_estudiante',
                           'docente_periodo':'periodo',
                           'docente_asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

id_docente,key_estudiante,periodo,asignatura,docente_-1,docente_1,docente_2,docente_3,docente_4,docente_5,docente_6,...,docente_20,docente_21,docente_22,docente_23,docente_24,docente_25,docente_26,docente_27,docente_28,docente_29
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [286]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no tomó la clase con ese docente.  1: si la tomó con ese docente

In [287]:
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,docente_20,docente_21,docente_22,docente_23,docente_24,docente_25,docente_26,docente_27,docente_28,docente_29
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ahora pivoteamos el **colegio** para hacer hot-enconding

In [288]:
temp = DATOS[['key_estudiante','periodo','asignatura','colegio']].copy()
temp['valor'] = 1
temp.head()

,key_estudiante,periodo,asignatura,colegio,valor
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,0047377,1
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,0047377,1
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,0203539,1
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,0206317,1
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,0047257,1


In [289]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='colegio', values='valor').reset_index()
prefijo = 'colegio_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

colegio,key_estudiante,periodo,asignatura,colegio_0040007,colegio_0040102,colegio_0040110,colegio_0040115,colegio_0040464,colegio_0040508,colegio_0040519,...,colegio_8957635,colegio_8959798,colegio_8966791,colegio_8966932,colegio_8967201,colegio_8969340,colegio_8971605,colegio_8982673,colegio_8985906,colegio_OTHR
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [290]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no tomó la clase con ese docente.  1: si la tomó con ese docente
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,colegio_8957635,colegio_8959798,colegio_8966791,colegio_8966932,colegio_8967201,colegio_8969340,colegio_8971605,colegio_8982673,colegio_8985906,colegio_OTHR
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [291]:
DATOS.shape

(7145, 694)

In [292]:
DATOS.to_pickle('checkPoints/DATOS0.pkl')

## **1. LABEL: calculamos si dada una fila en d de DATOS, si cancelo o perdió, o ganó**

dada una fila, miramos si está con estado D (DROP).  LABEL = 0.
Si está en estado E con nota < 30,     LABEL = 0
si esta en estado E con nota >= 3.0,   LABEL = 1
otro caso:  LABEL = -1

In [293]:
DATOS = pd.read_pickle('checkPoints/DATOS0.pkl')

In [294]:
def calcularLABEL(row):

    # Si el estado es cancelado (DROP) se retorna 1  (estos casos tienen nota nula)
    if row['estado'] == 'D':
        return 1 
    
    # si el estado es matriculado (ENROLL)
    if row['estado'] == 'E':

        # si la nota es nula (-1), es porque la está viendo y por tanto es una fila para TEST, no para TRAINING
        if row['nota']==-1:
            return -1
        # si la nota es menor a 3, la perdió y por tanto se marca con un 1
        if row['nota']<3.0:
            return 1
        else:
            # si no, no la perdió y por tanto se marca con un 0.
            return 0 

DATOS['LABEL'] = DATOS.apply(calcularLABEL, axis=1)
        

In [295]:
DATOS['LABEL'].value_counts()

LABEL
 0    4820
 1    1791
-1     534
Name: count, dtype: int64

Salvamos este punto como DATA1 como pickle

In [296]:
DATOS.to_pickle('checkPoints/DATOS1.pkl')

In [297]:
DATOS.shape

(7145, 695)

## **2. ICFES**

obtenemos las áreas del icfes y las adicionamos a DATA.
la tabla original venía con el nombre del examen (test_id) y el nombre de los componentees (ej: BIO).  sin embargo, aparecen algunos que sugieren que son items (ej: BIO1).  Hay también totales (TOT).

Para no eliminar nada, se trata cada uno como si fuera un item distinto, concatenando el **test_id** con el **test_component**.

In [298]:
DATOS = pd.read_pickle('checkPoints/DATOS1.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT  key_estudiante, CONCAT(test_id,'_',test_component) item, score
    FROM f_icfes
    ORDER BY test_id, CONCAT(test_id,'_',test_component);
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,item,score
0,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_APM,48.0
1,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_APV,58.0
2,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_BIO,59.0
3,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_COM,51.0
4,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_ELE,55.0


In [299]:
temp_pivot = temp.pivot(index=['key_estudiante'], columns='item', values='score').reset_index()
prefijo = 'icfes_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

item,key_estudiante,icfes_ICFES3_CCIUD,icfes_ICFES3_DCC,icfes_ICFES3_DCN,icfes_ICFES3_DIN,icfes_ICFES3_DLC,icfes_ICFES3_DMA,icfes_ICFES3_DRC,icfes_ICFES3_DSC,icfes_ICFES3_IPEM,...,icfes_ICF_2_QUIC1,icfes_ICF_2_QUIC2,icfes_ICF_2_QUIC3,icfes_ICF_2_QUIG1,icfes_ICF_2_QUIG2,icfes_ICF_2_QUIG3,icfes_ICF_2_QUIG4,icfes_ICF_2_QUIPR,icfes_ICF_2_SOC,icfes_ICF_2_TOT
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,007A5DF5947E08FEBA1AED5A442DE6199D907121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,007AD6584DA00DEE96BA57A369CDAB8C9C832D53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00844F609DA54DFF214529B66202E11E4D020184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [300]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante'], how='left')
DATOS = DATOS.fillna(-100) # -100: el icfes de ese estudiante no tiene dicho componente
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,icfes_ICF_2_QUIC1,icfes_ICF_2_QUIC2,icfes_ICF_2_QUIC3,icfes_ICF_2_QUIG1,icfes_ICF_2_QUIG2,icfes_ICF_2_QUIG3,icfes_ICF_2_QUIG4,icfes_ICF_2_QUIPR,icfes_ICF_2_SOC,icfes_ICF_2_TOT
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,8.0,8.2,6.1,9.6,5.8,7.2,7.4,0.0,-100.0,348.0


In [301]:
DATOS.to_pickle('checkPoints/DATOS2.pkl')

In [302]:
DATOS.shape

(7145, 818)

## **3. promedio nivel nota asignaturas en semestres anteriores**

Promedio nivel_nota obtenido por semestres anteriores.
Se calcula el promedio de nivel nota por semestre, se mira el orden de este y se calcula
la diferencia contra el orden del periodo actual


In [303]:
DATOS = pd.read_pickle('checkPoints/DATOS2.pkl')

# OJO:   Nivel nota tiene la siguiente informacion
# -10:   los estudiante que apenas están viendo la asignatura (periodo 20231)
# - 5:   los que la cancelaron
#   0    los que la perdieron
#   1:   los que la ganaron regularimbis
#   2:   los que la ganaron bien (igual o mayor a 3.8) 
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, 
           p.ordenRE - p2.ordenRE hace, AVG(ma2.nivel_nota) promedio
    FROM f_matriculasAsignaturas ma						-- tabla base
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.tipo = 'RE'									-- solo se miran asig anteriores vistas en períodos regulares
    AND p2.ordenRE < p.ordenRE                          -- solo miramos en el orden correcto
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, p.ordenRE - p2.ordenRE;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,hace,promedio
0,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20121,300MAG006,1.0,0.8333
1,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20121,300MAG006,2.0,1.3333
2,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20121,300MAG006,3.0,0.8333
3,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20121,300MAG006,5.0,1.0000
4,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20121,300MAG006,6.0,1.0000


In [304]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='hace', values='promedio').reset_index()
prefijo = 'prom_nivel_hace_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

hace,key_estudiante,periodo,asignatura,prom_nivel_hace_1.0,prom_nivel_hace_2.0,prom_nivel_hace_3.0,prom_nivel_hace_4.0,prom_nivel_hace_5.0,prom_nivel_hace_6.0,prom_nivel_hace_7.0,...,prom_nivel_hace_14.0,prom_nivel_hace_15.0,prom_nivel_hace_16.0,prom_nivel_hace_17.0,prom_nivel_hace_18.0,prom_nivel_hace_19.0,prom_nivel_hace_21.0,prom_nivel_hace_22.0,prom_nivel_hace_23.0,prom_nivel_hace_24.0
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,2.0000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,2.0000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,2.0000,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,2.0000,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,-0.1429,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [305]:
temp_pivot.shape

(6932, 26)

In [306]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no es posible calcular un periodo tan anterior (no estaba el estud aun)
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,prom_nivel_hace_14.0,prom_nivel_hace_15.0,prom_nivel_hace_16.0,prom_nivel_hace_17.0,prom_nivel_hace_18.0,prom_nivel_hace_19.0,prom_nivel_hace_21.0,prom_nivel_hace_22.0,prom_nivel_hace_23.0,prom_nivel_hace_24.0
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0


In [307]:
DATOS.to_pickle('checkPoints/DATOS3.pkl')
DATOS.shape


(7145, 841)

## **4. promedio nivel nota asignaturas por departamento, de asignaturas anteriores**

Promedio nivel_nota obtenido por semestres anteriores.
Se calcula el promedio de nivel nota por dpto, se mira el orden de este y se calcula
la diferencia contra el orden del periodo actual


In [308]:
DATOS = pd.read_pickle('checkPoints/DATOS3.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, 
           ma2.departamento, AVG(ma2.nivel_nota) promedio
    FROM f_matriculasAsignaturas ma						-- tabla base
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.orden < p.orden                          -- solo miramos en el orden correcto
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma2.departamento;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,departamento,promedio
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,CIP,2.0
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,CSP,2.0
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,EIH,2.0
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,EPG,2.0
4,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,IGE,2.0


In [309]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='departamento', values='promedio').reset_index()
prefijo = 'prom_nivel_dpto_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

departamento,key_estudiante,periodo,asignatura,prom_nivel_dpto_31,prom_nivel_dpto_AAF,prom_nivel_dpto_ADM,prom_nivel_dpto_AGF,prom_nivel_dpto_AGG,prom_nivel_dpto_ANB,prom_nivel_dpto_ANC,...,prom_nivel_dpto_MED,prom_nivel_dpto_NTD,prom_nivel_dpto_OBA,prom_nivel_dpto_PSC,prom_nivel_dpto_PSG,prom_nivel_dpto_PSV,prom_nivel_dpto_RLD,prom_nivel_dpto_SAP,prom_nivel_dpto_TEG,prom_nivel_dpto_TMS
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [310]:
temp_pivot.shape

(7081, 123)

In [311]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no es posible calcular dpto cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,prom_nivel_dpto_MED,prom_nivel_dpto_NTD,prom_nivel_dpto_OBA,prom_nivel_dpto_PSC,prom_nivel_dpto_PSG,prom_nivel_dpto_PSV,prom_nivel_dpto_RLD,prom_nivel_dpto_SAP,prom_nivel_dpto_TEG,prom_nivel_dpto_TMS
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-2.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,1.0,-10.0


In [312]:
print(DATOS.shape)
DATOS.to_pickle('checkPoints/DATOS4.pkl')

(7145, 961)


## **5. promedio nivel nota por asignaturas anteriores**

Promedio nivel_nota obtenido de semestres anteriores.
Se calcula el promedio de nivel nota por asignatra, se mira el orden de este y se calcula
la diferencia contra el orden del periodo actual.

Es el promedio pues pudo haberse visto una misma asigantura más de una vez

In [313]:
DATOS = pd.read_pickle('checkPoints/DATOS4.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, 
           ma2.asignatura asig_ant, AVG(ma2.nivel_nota) promedio
    FROM f_matriculasAsignaturas ma						-- tabla base
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.orden < p.orden                          -- solo miramos en el orden correcto
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma2.asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,asig_ant,promedio
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300CIP001,2.0
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300CSP003,2.0
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300EIH001,2.0
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300EPG002,2.0
4,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300IGE001,2.0


In [314]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='asig_ant', values='promedio').reset_index()
prefijo = 'prom_nivel_asig_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

asig_ant,key_estudiante,periodo,asignatura,prom_nivel_asig_300AAF001,prom_nivel_asig_300AAF002,prom_nivel_asig_300AAF003,prom_nivel_asig_300AAF004,prom_nivel_asig_300AAF006,prom_nivel_asig_300AAF007,prom_nivel_asig_300AAF008,...,prom_nivel_asig_300PSV002,prom_nivel_asig_300RLD001,prom_nivel_asig_300SAP004,prom_nivel_asig_300TEG001,prom_nivel_asig_300TEG002,prom_nivel_asig_300TMS001,prom_nivel_asig_300TMS002,prom_nivel_asig_300TMS006,prom_nivel_asig_303IBA002,prom_nivel_asig_MT131
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [315]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no es posible calcular asig cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,prom_nivel_asig_300PSV002,prom_nivel_asig_300RLD001,prom_nivel_asig_300SAP004,prom_nivel_asig_300TEG001,prom_nivel_asig_300TEG002,prom_nivel_asig_300TMS001,prom_nivel_asig_300TMS002,prom_nivel_asig_300TMS006,prom_nivel_asig_303IBA002,prom_nivel_asig_MT131
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,1.0,-5.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,-10.0,-10.0,-10.0,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0


In [316]:
print(DATOS.shape)
DATOS.to_pickle('checkPoints/DATOS5.pkl')

(7145, 1646)


## **6. promedio nivel nota por numero de creditos anteriores**

Promedio nivel_nota obtenido de semestres anteriores.
Se calcula el promedio de nivel nota por numero de creditos, 

Es el promedio pues pudo haberse visto una misma asigantura más de una vez

In [317]:
DATOS = pd.read_pickle('checkPoints/DATOS5.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, 
           ma2.creditos, AVG(ma2.nivel_nota) promedio
    FROM f_matriculasAsignaturas ma						-- tabla base
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.orden < p.orden                          -- solo miramos en el orden correcto, tanto CV como RE
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma2.creditos;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,creditos,promedio
0,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20202,300MAG006,3,0.5455
1,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20202,300MAG006,4,0.6667
2,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20202,300MAG006,2,1.4500
3,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20202,300MAG007,3,0.5455
4,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20202,300MAG007,4,0.6667


In [318]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='creditos', values='promedio').reset_index()
prefijo = 'prom_nivel_creditos_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

creditos,key_estudiante,periodo,asignatura,prom_nivel_creditos_0,prom_nivel_creditos_1,prom_nivel_creditos_2,prom_nivel_creditos_3,prom_nivel_creditos_4,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,2.0,2.0000,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,2.0,2.0000,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,2.0,2.0000,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,2.0,2.0000,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,1.1667,-0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [319]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no es posible calcular asig cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_2,prom_nivel_creditos_3,prom_nivel_creditos_4,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,1.4000,1.0000,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.5000,0.9091,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,1.3333,1.0000,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,1.0000,0.6667,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,1.2500,1.5000,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0


In [320]:
DATOS.to_pickle('checkPoints/DATOS6.pkl')
print(DATOS.shape)

(7145, 1658)


## **7. cantidad de cursos que ya había tomado el estudiante antes de tomar el curso actual**

se calcula el total de cursos que ya había tomado el estudiante antes de tomar este (sin contar los matriculados este semestre).  SE incluyen cursos tomados en CV y RE, y si repitió un curso se cuenta 2 veces o más

In [321]:
DATOS = pd.read_pickle('checkPoints/DATOS6.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, count(*) q_cursos_tomados
    FROM f_matriculasAsignaturas ma						-- asignaturas a tomar en cuenta
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.orden < p.orden     						-- solo se miran periodos anteriores
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,q_cursos_tomados
0,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20202,300MAG006,51
1,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20202,300MAG007,51
2,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20121,300MAG006,45
3,B4FEAEEBC40BBF5FE724465B5C672BF178DC5588,20192,300MAG006,58
4,B4FEAEEBC40BBF5FE724465B5C672BF178DC5588,20192,300MAG007,58


In [322]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_3,prom_nivel_creditos_4,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15,q_cursos_tomados
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,1.0000,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,13.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.9091,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,21.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,1.0000,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.6667,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,1.5000,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0


In [323]:
DATOS.to_pickle('checkPoints/DATOS7.pkl')
print(DATOS.shape)

(7145, 1659)


## **8. cantidad de semestres regulares que ya lleva el estudiante en la U al tomar el curso**

se calcula el total periodos regulares que lleva el estudiante en la U antes de tomar el curso

In [324]:
DATOS = pd.read_pickle('checkPoints/DATOS7.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT key_estudiante, periodo, asignatura, count(*) q_semestres_reg_tomados
    FROM (
        SELECT DISTINCT ma.key_estudiante, ma.periodo, ma.asignatura, ma2.periodo periodo_ant
        FROM f_matriculasAsignaturas ma						-- asignaturas a tomar en cuenta
        INNER JOIN f_periodos p								--  periodo de ma
        ON p.periodo = ma.periodo
        INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
        ON  ma2.key_estudiante = ma.key_estudiante
        INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
        ON p2.periodo = ma2.periodo
        WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
        AND p2.orden < p.orden     						-- solo se miran periodos anteriores
    ) V
    GROUP BY key_estudiante, periodo, asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,q_semestres_reg_tomados
0,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20202,300MAG006,8
1,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20202,300MAG007,8
2,401B4BCFA7ACB3FF1B0D4833DBEAC02E778D0811,20121,300MAG006,7
3,B4FEAEEBC40BBF5FE724465B5C672BF178DC5588,20192,300MAG006,10
4,B4FEAEEBC40BBF5FE724465B5C672BF178DC5588,20192,300MAG007,10


In [325]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_4,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15,q_cursos_tomados,q_semestres_reg_tomados
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,13.0,2.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,21.0,4.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0


In [326]:
DATOS.to_pickle('checkPoints/DATOS8.pkl')
print(DATOS.shape)

(7145, 1660)


## **9. cantidad de cursos que tomó en cursos de verano**

se calcula de cursos que tomó en períodos CV anteriore al curso actual

In [327]:
DATOS = pd.read_pickle('checkPoints/DATOS8.pkl')

query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, count(*) q_cursos_CV
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.key_estudiante = ma.key_estudiante
    INNER JOIN  f_periodos p2
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1						-- tomamos en cuenta solo las asignaturas de interés
    AND p2.tipo ='CV'							-- los periodos anteriores solo se toman en cuenta los tipo CV
    AND p2.orden < p.orden						-- solo tomamos en cuenta periodos anteriores al periodo de la asig
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura;
    """
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,q_cursos_CV
0,6037D3BECE0B7AA9162CCC7DCB40D137F72A08BB,20152,300MAG006,1
1,6037D3BECE0B7AA9162CCC7DCB40D137F72A08BB,20152,300MAG007,1
2,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,1
3,6037D3BECE0B7AA9162CCC7DCB40D137F72A08BB,20161,300MAG007,1
4,306023AB9393647A8E4AD19E67EC6AB620240D78,20161,300MAG007,1


In [328]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores tipo CV
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15,q_cursos_tomados,q_semestres_reg_tomados,q_cursos_CV
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,13.0,2.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,21.0,4.0,1.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0,0.0


In [329]:
DATOS.to_pickle('checkPoints/DATOS9.pkl')
print(DATOS.shape)

(7145, 1661)


## **10. cantidad de cursos que tomó en cursos de verano por cpto**

se calcula de cursos que tomó en períodos CV anteriore al curso actual, por dpto

In [330]:
DATOS = pd.read_pickle('checkPoints/DATOS9.pkl')

query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, ma2.departamento, count(*) q_cursos_CV
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.key_estudiante = ma.key_estudiante
    INNER JOIN  f_periodos p2
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1						-- tomamos en cuenta solo las asignaturas de interés
    AND p2.tipo ='CV'							-- los periodos anteriores solo se toman en cuenta los tipo CV
    AND p2.orden < p.orden						-- solo tomamos en cuenta periodos anteriores al periodo de la asig
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma2.departamento;
    """
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,departamento,q_cursos_CV
0,6037D3BECE0B7AA9162CCC7DCB40D137F72A08BB,20152,300MAG006,MAG,1
1,6037D3BECE0B7AA9162CCC7DCB40D137F72A08BB,20152,300MAG007,MAG,1
2,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,MAG,1
3,6037D3BECE0B7AA9162CCC7DCB40D137F72A08BB,20161,300MAG007,MAG,1
4,306023AB9393647A8E4AD19E67EC6AB620240D78,20161,300MAG007,MAG,1


In [331]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='departamento', values='q_cursos_CV').reset_index()
prefijo = 'q_cursos_CV_dpto_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

departamento,key_estudiante,periodo,asignatura,q_cursos_CV_dpto_AQA,q_cursos_CV_dpto_LEI,q_cursos_CV_dpto_MAE,q_cursos_CV_dpto_MAG,q_cursos_CV_dpto_MDM
0,00844F609DA54DFF214529B66202E11E4D020184,20201,300MAG006,NaN,NaN,NaN,1.0,NaN
1,00844F609DA54DFF214529B66202E11E4D020184,20201,300MAG007,NaN,NaN,NaN,1.0,NaN
2,00844F609DA54DFF214529B66202E11E4D020184,20211,300MAG006,NaN,NaN,NaN,1.0,NaN
3,00844F609DA54DFF214529B66202E11E4D020184,20211,300MAG007,NaN,NaN,NaN,1.0,NaN
4,0163D087A12913268CBEB4C483BE1F6B6A4EB7F2,20222,300MAG006,NaN,NaN,NaN,1.0,NaN


In [332]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores tipo CV
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_12,prom_nivel_creditos_15,q_cursos_tomados,q_semestres_reg_tomados,q_cursos_CV,q_cursos_CV_dpto_AQA,q_cursos_CV_dpto_LEI,q_cursos_CV_dpto_MAE,q_cursos_CV_dpto_MAG,q_cursos_CV_dpto_MDM
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,-10.0,-10.0,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,-10.0,-10.0,21.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-10.0,-10.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-10.0,-10.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,-10.0,-10.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [333]:
DATOS.to_pickle('checkPoints/DATOS10.pkl')
print(DATOS.shape)

(7145, 1666)


## **11. cuales cursos  tomó en cursos de verano por profe**

lista de cursos tomados en curso de verano por profe.  el valor es la cantidad (se puede repetir asig profe más de una vez)

In [334]:
DATOS = pd.read_pickle('checkPoints/DATOS10.pkl')

query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura,  d.id_docente, count(*) cuantos
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.key_estudiante = ma.key_estudiante
    INNER JOIN  f_periodos p2
    ON p2.periodo = ma2.periodo
    INNER JOIN f_docentes d
    ON d.key_docente = ma2.key_docente
    WHERE ma.analizar = 1						-- tomamos en cuenta solo las asignaturas de interés
    AND p2.tipo ='CV'							-- los periodos anteriores solo se toman en cuenta los tipo CV
    AND p2.orden < p.orden						-- solo tomamos en cuenta periodos anteriores al periodo de la asig
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, d.id_docente;
    """
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,id_docente,cuantos
0,EF8F28CA573273EF9B64853FF1A3D931BBAB9D73,20201CV,300MAG007,14,2
1,EF8F28CA573273EF9B64853FF1A3D931BBAB9D73,20192,300MAG006,14,2
2,EF8F28CA573273EF9B64853FF1A3D931BBAB9D73,20192,300MAG007,14,2
3,EF8F28CA573273EF9B64853FF1A3D931BBAB9D73,20191,300MAG006,14,2
4,EF8F28CA573273EF9B64853FF1A3D931BBAB9D73,20191,300MAG007,14,2


In [335]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='id_docente', values='cuantos').reset_index()
prefijo = 'q_cursos_CV_docente_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

id_docente,key_estudiante,periodo,asignatura,q_cursos_CV_docente_14,q_cursos_CV_docente_16,q_cursos_CV_docente_20,q_cursos_CV_docente_22,q_cursos_CV_docente_24,q_cursos_CV_docente_29
0,03E1E4836166AC1FD3C2F0E415F3578D91656E35,20231,300MAG006,NaN,NaN,NaN,NaN,NaN,2.0
1,03E1E4836166AC1FD3C2F0E415F3578D91656E35,20231,300MAG007,NaN,NaN,NaN,NaN,NaN,2.0
2,0642AA8AE9E5CA66990F2427D0CDA2477972BB2C,20191,300MAG006,NaN,NaN,1.0,NaN,NaN,NaN
3,0642AA8AE9E5CA66990F2427D0CDA2477972BB2C,20191,300MAG007,NaN,NaN,1.0,NaN,NaN,NaN
4,2338F23C514CFEBC29C100981B55DB7B41624778,20191,300MAG006,NaN,NaN,1.0,NaN,NaN,NaN


In [336]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores tipo CV
DATOS.to_pickle('checkPoints/DATOS11.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1672)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,q_cursos_CV_dpto_LEI,q_cursos_CV_dpto_MAE,q_cursos_CV_dpto_MAG,q_cursos_CV_dpto_MDM,q_cursos_CV_docente_14,q_cursos_CV_docente_16,q_cursos_CV_docente_20,q_cursos_CV_docente_22,q_cursos_CV_docente_24,q_cursos_CV_docente_29
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **12. promedio semestre RE hace**

Promedio de semestres regulares anteriores.   
OJO hay que tener cuidado.   Si solo quiero calcular distancias contra periodos regulares, hay que entender que la fila f_matriculasAsignaturas que se analiza puede ser RE o CV.  los CV no tienen ordenRE y por tanto no puede calcularse la distancia.

Si se quiere calcular distancias contra RE, colocar la condicion de RE para periodos de f_matriculasAsignaturas, pero usar orden y no ordenRE pues si la fila de f_matriculasAsignatura es CV no va a tener orderRE (será NULO) y no podrá calcularse las distancia sy seguro saldrna filas repetidas.



In [337]:
DATOS = pd.read_pickle('checkPoints/DATOS11.pkl')

query = """
    SELECT  ma.key_estudiante, ma.periodo, ma.asignatura,
            p.orden - p2.orden hace, m.promedio_semestre
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculas m
    ON m.key_estudiante = ma.key_estudiante
    INNER JOIN  f_periodos p2
    ON p2.periodo = m.periodo
    WHERE ma.analizar = 1						-- tomamos en cuenta solo las asignaturas de interés
    AND p2.orden < p.orden;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,hace,promedio_semestre
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,1,3.64
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,3,3.64
2,306023AB9393647A8E4AD19E67EC6AB620240D78,20161,300MAG007,4,3.64
3,306023AB9393647A8E4AD19E67EC6AB620240D78,20162,300MAG007,6,3.64
4,306023AB9393647A8E4AD19E67EC6AB620240D78,20171,300MAG007,8,3.64


In [338]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], 
                        columns='hace', values='promedio_semestre').reset_index()
prefijo = 'promedio_sem_hace_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

hace,key_estudiante,periodo,asignatura,promedio_sem_hace_1,promedio_sem_hace_2,promedio_sem_hace_3,promedio_sem_hace_4,promedio_sem_hace_5,promedio_sem_hace_6,promedio_sem_hace_7,...,promedio_sem_hace_24,promedio_sem_hace_25,promedio_sem_hace_27,promedio_sem_hace_28,promedio_sem_hace_29,promedio_sem_hace_30,promedio_sem_hace_32,promedio_sem_hace_33,promedio_sem_hace_34,promedio_sem_hace_35
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,4.78,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,4.78,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,4.79,NaN,4.78,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,4.79,NaN,4.78,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,2.23,NaN,2.94,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no existe la distancia hasta un RE (ej: el primer semestre)
DATOS.to_pickle('checkPoints/DATOS12.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1705)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,promedio_sem_hace_24,promedio_sem_hace_25,promedio_sem_hace_27,promedio_sem_hace_28,promedio_sem_hace_29,promedio_sem_hace_30,promedio_sem_hace_32,promedio_sem_hace_33,promedio_sem_hace_34,promedio_sem_hace_35
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0


## **13. tamaño del curso actual**

la data que me dieron permite calcular los tamaños de los grupos de 300MAG006 o 300MAG007 pero no de los otros.   este punto por tanto si es calculable.

El query se hace incluyendo el grupo, pero este no se usa al insertar la variable cuantos dentro de DATOS.   Confirmar que sigue solo habiendo un UK por key_estudiante, periodo, asignatura.  ESto se confirmó en consultasDATOS.SQL


In [340]:
DATOS = pd.read_pickle('checkPoints/DATOS12.pkl')

query = """ 
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo, COUNT(*) curso_actual_tamano
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.periodo = ma.periodo
    AND ma2.asignatura = ma.asignatura
    AND ma2.grupo = ma.grupo
    WHERE ma.analizar = 1
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,grupo,curso_actual_tamano
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,2432,30
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,2432,30
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,1324,35
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,1324,35
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,1297,27


In [341]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-0) # 0: aunque no debería existir
DATOS.to_pickle('checkPoints/DATOS13.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1707)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,promedio_sem_hace_27,promedio_sem_hace_28,promedio_sem_hace_29,promedio_sem_hace_30,promedio_sem_hace_32,promedio_sem_hace_33,promedio_sem_hace_34,promedio_sem_hace_35,grupo,curso_actual_tamano
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,1895,2
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,3402,1
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,4577,3
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,4576,2
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,4578,1


## **14. composicion del curso actual por programa**

para determinar heterogeniedad, hay que incluir el primer programa del estudainte en DATOS, y también la composición del grupo por primer_programa

In [342]:
DATOS = pd.read_pickle('checkPoints/DATOS13.pkl')

query = """ 
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, e.primer_programa estudiante_programa
    FROM f_matriculasAsignaturas ma
    INNER JOIN  f_estudiantes e
    ON e.key_estudiante = ma.key_estudiante
    WHERE ma.analizar = 1;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,estudiante_programa
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,30
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,30
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,30
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,30
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,20


In [343]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: aunque no debería existir
print(DATOS.shape)
DATOS.head()

(7145, 1708)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,promedio_sem_hace_28,promedio_sem_hace_29,promedio_sem_hace_30,promedio_sem_hace_32,promedio_sem_hace_33,promedio_sem_hace_34,promedio_sem_hace_35,grupo,curso_actual_tamano,estudiante_programa
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,1895,2,20
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,3402,1,20
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,4577,3,71
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,4576,2,71
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,4578,1,10


In [344]:
query = """ 
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo, e.primer_programa programa, COUNT(*) cuantos
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.periodo = ma.periodo
    AND ma2.asignatura = ma.asignatura
    AND ma2.grupo = ma.grupo
    INNER JOIN f_estudiantes e
    ON e.key_estudiante = ma.key_estudiante
    WHERE ma.analizar = 1
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo, e.primer_programa
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,grupo,programa,cuantos
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,2432,30,30
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,2432,30,30
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,1324,30,35
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,1324,30,35
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,1297,20,27


In [345]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], 
                        columns='programa', values='cuantos').reset_index()
prefijo = 'curso_actual_prog_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

programa,key_estudiante,periodo,asignatura,curso_actual_prog_10,curso_actual_prog_10001,curso_actual_prog_10002,curso_actual_prog_10003,curso_actual_prog_10004,curso_actual_prog_11002,curso_actual_prog_12003,curso_actual_prog_19002,curso_actual_prog_19003,curso_actual_prog_20,curso_actual_prog_30,curso_actual_prog_40,curso_actual_prog_51,curso_actual_prog_68,curso_actual_prog_70,curso_actual_prog_71
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN


In [346]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no hay estudiantes de ese programa en el curso
DATOS.to_pickle('checkPoints/DATOS14.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1724)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,curso_actual_prog_12003,curso_actual_prog_19002,curso_actual_prog_19003,curso_actual_prog_20,curso_actual_prog_30,curso_actual_prog_40,curso_actual_prog_51,curso_actual_prog_68,curso_actual_prog_70,curso_actual_prog_71
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **15. composicion del curso actual por edades**

para determinar heterogeniedad, hay que incluir la distribución por edades.  ya se incluyó al principio la edad del estudiante

In [347]:
DATOS = pd.read_pickle('checkPoints/DATOS14.pkl')

query = """ 
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo, 
          TIMESTAMPDIFF(YEAR, e.birthdate, p.fecha_inicio)  edad, count(*) cuantos
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.periodo = ma.periodo
    AND ma2.asignatura = ma.asignatura
    AND ma2.grupo = ma.grupo
    INNER JOIN f_estudiantes e
    ON e.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    WHERE ma.analizar = 1
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo, TIMESTAMPDIFF(YEAR, e.birthdate, p.fecha_inicio)
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,grupo,edad,cuantos
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,2432,17,30
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,2432,17,30
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,1324,18,35
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,1324,18,35
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,1297,21,27


In [348]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], 
                        columns='edad', values='cuantos').reset_index()
prefijo = 'curso_actual_edad_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

edad,key_estudiante,periodo,asignatura,curso_actual_edad_15,curso_actual_edad_16,curso_actual_edad_17,curso_actual_edad_18,curso_actual_edad_19,curso_actual_edad_20,curso_actual_edad_21,...,curso_actual_edad_24,curso_actual_edad_25,curso_actual_edad_26,curso_actual_edad_27,curso_actual_edad_28,curso_actual_edad_29,curso_actual_edad_30,curso_actual_edad_34,curso_actual_edad_37,curso_actual_edad_41
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,30.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,30.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,35.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,35.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [349]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no hay estudiantes de esa edad en el curso
DATOS.to_pickle('checkPoints/DATOS15.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1743)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,curso_actual_edad_24,curso_actual_edad_25,curso_actual_edad_26,curso_actual_edad_27,curso_actual_edad_28,curso_actual_edad_29,curso_actual_edad_30,curso_actual_edad_34,curso_actual_edad_37,curso_actual_edad_41
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **16. composicion del curso actual por genero**

para determinar heterogeniedad, hay que incluir la distribución por genero.  ya se incluyó al principio el genero (estudiante_genero) del estudiante

In [350]:
DATOS = pd.read_pickle('checkPoints/DATOS15.pkl')

query = """ 
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo, 
          e.sex, count(*) cuantos
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.periodo = ma.periodo
    AND ma2.asignatura = ma.asignatura
    AND ma2.grupo = ma.grupo
    INNER JOIN f_estudiantes e
    ON e.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    WHERE ma.analizar = 1
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo, e.sex
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,grupo,sex,cuantos
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,2432,0,30
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,2432,0,30
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,1324,0,35
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,1324,0,35
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,1297,1,27


In [351]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], 
                        columns='sex', values='cuantos').reset_index()
prefijo = 'curso_actual_genero_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

sex,key_estudiante,periodo,asignatura,curso_actual_genero_0,curso_actual_genero_1
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,30.0,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,30.0,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,35.0,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,35.0,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,27.0


In [352]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no hay estudiantes de esa genero en el curso
DATOS.to_pickle('checkPoints/DATOS16.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1745)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,curso_actual_edad_26,curso_actual_edad_27,curso_actual_edad_28,curso_actual_edad_29,curso_actual_edad_30,curso_actual_edad_34,curso_actual_edad_37,curso_actual_edad_41,curso_actual_genero_0,curso_actual_genero_1
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## **17. carga semestre actual del estudiante**

para determinar la carga, miramos cuantas asignaturas está matriculando ese estudiante en este momento

In [353]:
DATOS = pd.read_pickle('checkPoints/DATOS16.pkl')

query = """ 
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, count(*) q_cursos_actuales
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_matriculasAsignaturas ma2
	ON ma2.key_estudiante = ma.key_estudiante
    AND ma2.periodo = ma.periodo
    WHERE ma.analizar = 1
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,q_cursos_actuales
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,8
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,8
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,6
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,6
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,8


In [354]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no hay estudiantes de esa genero en el curso
DATOS.to_pickle('checkPoints/DATOS17.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1746)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,curso_actual_edad_27,curso_actual_edad_28,curso_actual_edad_29,curso_actual_edad_30,curso_actual_edad_34,curso_actual_edad_37,curso_actual_edad_41,curso_actual_genero_0,curso_actual_genero_1,q_cursos_actuales
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,7
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,7
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,7
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7


## **18. carga semestre actual del estudiante por dpto**

para determinar la carga, miramos cuantas asignaturas está matriculando ese estudiante en este momento
por depto

In [6]:
DATOS = pd.read_pickle('checkPoints/DATOS17.pkl')

query = """ 
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, ma2.departamento, count(*) cuantos
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_matriculasAsignaturas ma2
	ON ma2.key_estudiante = ma.key_estudiante
    AND ma2.periodo = ma.periodo
    WHERE ma.analizar = 1
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo, ma2.departamento
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,departamento,cuantos
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,EIH,1
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,IGE,2
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,LEI,1
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,MAG,3
4,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,TEG,1


In [7]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], 
                        columns='departamento', values='cuantos').reset_index()
prefijo = 'Q_cursos_actuales_dpto_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

departamento,key_estudiante,periodo,asignatura,Q_cursos_actuales_dpto_31,Q_cursos_actuales_dpto_AAF,Q_cursos_actuales_dpto_ADM,Q_cursos_actuales_dpto_AGF,Q_cursos_actuales_dpto_ANB,Q_cursos_actuales_dpto_ANC,Q_cursos_actuales_dpto_AND,...,Q_cursos_actuales_dpto_MEA,Q_cursos_actuales_dpto_NTD,Q_cursos_actuales_dpto_OBA,Q_cursos_actuales_dpto_OCS,Q_cursos_actuales_dpto_PSC,Q_cursos_actuales_dpto_PSG,Q_cursos_actuales_dpto_RLD,Q_cursos_actuales_dpto_SAP,Q_cursos_actuales_dpto_TEG,Q_cursos_actuales_dpto_TMS
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no esta tomando cursos de ese dpto este semestre
DATOS.to_pickle('checkPoints/DATOS18.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1857)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,Q_cursos_actuales_dpto_MEA,Q_cursos_actuales_dpto_NTD,Q_cursos_actuales_dpto_OBA,Q_cursos_actuales_dpto_OCS,Q_cursos_actuales_dpto_PSC,Q_cursos_actuales_dpto_PSG,Q_cursos_actuales_dpto_RLD,Q_cursos_actuales_dpto_SAP,Q_cursos_actuales_dpto_TEG,Q_cursos_actuales_dpto_TMS
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## **19. carga semestre actual del estudiante por creditos**

para determinar la carga, miramos cuantas asignaturas está matriculando ese estudiante en este momento
por creditos

In [9]:
DATOS = pd.read_pickle('checkPoints/DATOS18.pkl')

query = """ 
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, ma2.creditos, count(*) cuantos
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_matriculasAsignaturas ma2
	ON ma2.key_estudiante = ma.key_estudiante
    AND ma2.periodo = ma.periodo
    WHERE ma.analizar = 1
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo, ma2.creditos
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura, ma.grupo;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,creditos,cuantos
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,2,4
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,3,4
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,2,4
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,3,4
4,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,3,4


In [10]:
temp.shape

(18781, 5)

In [11]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], 
                        columns='creditos', values='cuantos').reset_index()
prefijo = 'Q_cursos_actuales_creditos_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

creditos,key_estudiante,periodo,asignatura,Q_cursos_actuales_creditos_0,Q_cursos_actuales_creditos_1,Q_cursos_actuales_creditos_2,Q_cursos_actuales_creditos_3,Q_cursos_actuales_creditos_4,Q_cursos_actuales_creditos_6,Q_cursos_actuales_creditos_9,Q_cursos_actuales_creditos_12
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,3.0,5.0,NaN,NaN,NaN,NaN


In [12]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no esta tomando cursos de ese creditos este semestre
DATOS.to_pickle('checkPoints/DATOS19.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1865)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,Q_cursos_actuales_dpto_TEG,Q_cursos_actuales_dpto_TMS,Q_cursos_actuales_creditos_0,Q_cursos_actuales_creditos_1,Q_cursos_actuales_creditos_2,Q_cursos_actuales_creditos_3,Q_cursos_actuales_creditos_4,Q_cursos_actuales_creditos_6,Q_cursos_actuales_creditos_9,Q_cursos_actuales_creditos_12
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,2.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,1.0,0.0,0.0,0.0,4.0,2.0,1.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,1.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,1.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0


## **20. cuantas veces ya había visto la asig que está matriculando**

se calcula cuantas veces había matriculado la asig que está matriclando, sea en RE o CV

In [13]:
DATOS = pd.read_pickle('checkPoints/DATOS19.pkl')

query = """ 
	SELECT ma.key_estudiante, ma.periodo, ma.asignatura, count(*) cursos_per_canc_antes
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculasAsignaturas ma2
	ON ma2.key_estudiante = ma.key_estudiante
    AND ma2.asignatura = ma.asignatura
    INNER JOIN f_periodos p2
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1
    AND p2.orden < p.orden 
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,cursos_per_canc_antes
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,1
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,1
2,007A5DF5947E08FEBA1AED5A442DE6199D907121,20222,300MAG006,1
3,007A5DF5947E08FEBA1AED5A442DE6199D907121,20222,300MAG007,1
4,007AD6584DA00DEE96BA57A369CDAB8C9C832D53,20182,300MAG006,1


In [14]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no esta tomando cursos de ese creditos este semestre
DATOS.to_pickle('checkPoints/DATOS20.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1866)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,Q_cursos_actuales_dpto_TMS,Q_cursos_actuales_creditos_0,Q_cursos_actuales_creditos_1,Q_cursos_actuales_creditos_2,Q_cursos_actuales_creditos_3,Q_cursos_actuales_creditos_4,Q_cursos_actuales_creditos_6,Q_cursos_actuales_creditos_9,Q_cursos_actuales_creditos_12,cursos_per_canc_antes
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0


## **21. cuantas asig había perdido o cancelado antes de tomar esta asig**

se calcula cuantas veces había perdido o cancelado en periodos anteriores

In [15]:
DATOS = pd.read_pickle('checkPoints/DATOS20.pkl')

query = """ 
	SELECT ma.key_estudiante, ma.periodo, ma.asignatura, count(*) cursos_per_canc_antes_otros
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculasAsignaturas ma2
	ON ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1
    AND p2.orden < p.orden 
    AND ma2.nivel_nota in (-5,0)                               -- la canceló o la perdió
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,cursos_per_canc_antes_otros
0,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,5
1,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG007,5
2,007A5DF5947E08FEBA1AED5A442DE6199D907121,20221,300MAG006,1
3,007A5DF5947E08FEBA1AED5A442DE6199D907121,20221,300MAG007,1
4,007A5DF5947E08FEBA1AED5A442DE6199D907121,20222,300MAG006,3


In [16]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no esta tomando cursos de ese creditos este semestre
DATOS.to_pickle('checkPoints/DATOS21.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1867)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,Q_cursos_actuales_creditos_0,Q_cursos_actuales_creditos_1,Q_cursos_actuales_creditos_2,Q_cursos_actuales_creditos_3,Q_cursos_actuales_creditos_4,Q_cursos_actuales_creditos_6,Q_cursos_actuales_creditos_9,Q_cursos_actuales_creditos_12,cursos_per_canc_antes,cursos_per_canc_antes_otros
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,3.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0


## **22. cuantas asig había perdido o cancelado por dpto antes de tomar esta asig**

se calcula cuantas veces había perdido o cancelado en periodos anteriores por dpto

In [17]:
DATOS = pd.read_pickle('checkPoints/DATOS21.pkl')

query = """ 
	SELECT ma.key_estudiante, ma.periodo, ma.asignatura, ma2.departamento,count(*) cuantos
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculasAsignaturas ma2
	ON ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1
    AND p2.orden < p.orden 
    AND ma2.nivel_nota in (-5,0)                               -- la canceló o la perdió
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma2.departamento
    ORDER BY ma.key_estudiante, ma.periodo, ma.asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,departamento,cuantos
0,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,CIP,1
1,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,IGC,1
2,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,MAG,3
3,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG007,CIP,1
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG007,IGC,1


In [18]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], 
                        columns='departamento', values='cuantos').reset_index()
prefijo = 'cursos_per_canc_antes_otros_dpto_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

departamento,key_estudiante,periodo,asignatura,cursos_per_canc_antes_otros_dpto_31,cursos_per_canc_antes_otros_dpto_AAF,cursos_per_canc_antes_otros_dpto_ADM,cursos_per_canc_antes_otros_dpto_AGF,cursos_per_canc_antes_otros_dpto_ANB,cursos_per_canc_antes_otros_dpto_ANC,cursos_per_canc_antes_otros_dpto_AND,...,cursos_per_canc_antes_otros_dpto_MAA,cursos_per_canc_antes_otros_dpto_MAE,cursos_per_canc_antes_otros_dpto_MAG,cursos_per_canc_antes_otros_dpto_MDM,cursos_per_canc_antes_otros_dpto_MEA,cursos_per_canc_antes_otros_dpto_PSC,cursos_per_canc_antes_otros_dpto_PSG,cursos_per_canc_antes_otros_dpto_PSV,cursos_per_canc_antes_otros_dpto_TEG,cursos_per_canc_antes_otros_dpto_TMS
0,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,007A5DF5947E08FEBA1AED5A442DE6199D907121,20221,300MAG006,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,007A5DF5947E08FEBA1AED5A442DE6199D907121,20221,300MAG007,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,007A5DF5947E08FEBA1AED5A442DE6199D907121,20222,300MAG006,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no esta tomando cursos de ese creditos este semestre
DATOS.to_pickle('checkPoints/DATOS22.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1955)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,cursos_per_canc_antes_otros_dpto_MAA,cursos_per_canc_antes_otros_dpto_MAE,cursos_per_canc_antes_otros_dpto_MAG,cursos_per_canc_antes_otros_dpto_MDM,cursos_per_canc_antes_otros_dpto_MEA,cursos_per_canc_antes_otros_dpto_PSC,cursos_per_canc_antes_otros_dpto_PSG,cursos_per_canc_antes_otros_dpto_PSV,cursos_per_canc_antes_otros_dpto_TEG,cursos_per_canc_antes_otros_dpto_TMS
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **23. experiencia docente en este curso**

cuantas veces había dictado antes este curso este docente al momento que el estudiante matricula la asignatura.  Se usa la tabla calculada F_docentesAsignaturas que permite saber qué grupo dictó qué docente.  La pk es periodo, asignatura, grupo

In [20]:
DATOS = pd.read_pickle('checkPoints/DATOS22.pkl')

query = """ 
    SELECT  ma.key_estudiante, ma.periodo, ma.asignatura,  COUNT(*) docente_experencia_curso
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_docentesAsignaturas da
    ON da.key_docente = ma.key_docente		-- el mismo docente
    AND da.asignatura = ma.asignatura		-- la misma asignatura
    WHERE ma.analizar = 1
    AND da.orden < p.orden					-- el docente la dictó en periodos anteriores
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, da.id_docente;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,docente_experencia_curso
0,4AA557C1AF2286EB9378B0AFF2F967396E25DE88,20231,300MAG006,22
1,59238206E187BF2E512760EEB01C2C4CE162E02E,20231,300MAG006,22
2,210719A2C90CEF273556E78AA229415A33033460,20231,300MAG006,22
3,68E15608F4FB45EFD8413DFBDF767A54FE20F69B,20231,300MAG006,22
4,12162D5F3C25F98188ACE25C4D293187181A227F,20231,300MAG006,22


In [21]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: ese docente no tiene experiencia en ese curso 
DATOS.to_pickle('checkPoints/DATOS23.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1956)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,cursos_per_canc_antes_otros_dpto_MAE,cursos_per_canc_antes_otros_dpto_MAG,cursos_per_canc_antes_otros_dpto_MDM,cursos_per_canc_antes_otros_dpto_MEA,cursos_per_canc_antes_otros_dpto_PSC,cursos_per_canc_antes_otros_dpto_PSG,cursos_per_canc_antes_otros_dpto_PSV,cursos_per_canc_antes_otros_dpto_TEG,cursos_per_canc_antes_otros_dpto_TMS,docente_experencia_curso
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **24. experiencia docente en todos los curso**

cuantas veces había dictado antes cualquier curso este docente al momento que el estudiante matricula la asignatura.  Se usa la tabla calculada F_docentesAsignaturas que permite saber qué grupo dictó qué docente.  La pk es periodo, asignatura, grupo

In [22]:
DATOS = pd.read_pickle('checkPoints/DATOS23.pkl')

query = """ 
    SELECT  ma.key_estudiante, ma.periodo, ma.asignatura,  COUNT(*) docente_experencia_otros_cursos
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_docentesAsignaturas da
    ON da.key_docente = ma.key_docente		-- el mismo docente
    WHERE ma.analizar = 1
    AND da.orden < p.orden					-- el docente la dictó en periodos anteriores
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, da.id_docente;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,docente_experencia_otros_cursos
0,4AA557C1AF2286EB9378B0AFF2F967396E25DE88,20231,300MAG006,44
1,59238206E187BF2E512760EEB01C2C4CE162E02E,20231,300MAG006,44
2,210719A2C90CEF273556E78AA229415A33033460,20231,300MAG006,44
3,68E15608F4FB45EFD8413DFBDF767A54FE20F69B,20231,300MAG006,44
4,12162D5F3C25F98188ACE25C4D293187181A227F,20231,300MAG006,44


In [23]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: ese docente no tiene experiencia en ese curso 
DATOS.to_pickle('checkPoints/DATOS24.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1957)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,cursos_per_canc_antes_otros_dpto_MAG,cursos_per_canc_antes_otros_dpto_MDM,cursos_per_canc_antes_otros_dpto_MEA,cursos_per_canc_antes_otros_dpto_PSC,cursos_per_canc_antes_otros_dpto_PSG,cursos_per_canc_antes_otros_dpto_PSV,cursos_per_canc_antes_otros_dpto_TEG,cursos_per_canc_antes_otros_dpto_TMS,docente_experencia_curso,docente_experencia_otros_cursos
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **25. cursos tomados previamente con el mismo docente (lo conoce)**

cuantas veces habia tomado clases con ese docente el estudiante (en cualquier asignatura)

In [24]:
DATOS = pd.read_pickle('checkPoints/DATOS24.pkl')

query = """ 
    SELECT  ma.key_estudiante, ma.periodo, ma.asignatura, COUNT(*) cursos_tomados_antes_con_docente
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2
    ON ma2.key_estudiante = ma.key_estudiante			-- el mismo estudiante
    AND ma2.key_docente = ma.key_docente				-- el mismo docente
    INNER JOIN f_periodos p2
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- solo filas de interés
    AND p2.orden < p.orden								-- periodos anteriores
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,cursos_tomados_antes_con_docente
0,AD60843C610D605B14A18492B1E7E42AA11D3ABD,20202,300MAG006,2
1,AD60843C610D605B14A18492B1E7E42AA11D3ABD,20202,300MAG007,2
2,973B2D227F2BC391A412ACFDA6DEEF5B14678DDF,20201,300MAG006,6
3,973B2D227F2BC391A412ACFDA6DEEF5B14678DDF,20201,300MAG007,6
4,B827E2B2BB13D1EE7D6E06A017F74662EDA9BD6B,20192,300MAG006,2


In [25]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no habia tomado cursos con ese docente antes
DATOS.to_pickle('checkPoints/DATOS25.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1958)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,cursos_per_canc_antes_otros_dpto_MDM,cursos_per_canc_antes_otros_dpto_MEA,cursos_per_canc_antes_otros_dpto_PSC,cursos_per_canc_antes_otros_dpto_PSG,cursos_per_canc_antes_otros_dpto_PSV,cursos_per_canc_antes_otros_dpto_TEG,cursos_per_canc_antes_otros_dpto_TMS,docente_experencia_curso,docente_experencia_otros_cursos,cursos_tomados_antes_con_docente
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **26. promedio nivel_nota obtenido con ese docente**

cual es el promedio de nivel_nota que el estudiante ha obtenido en pasados cursos con ese docente

In [26]:
DATOS = pd.read_pickle('checkPoints/DATOS25.pkl')

query = """ 
    SELECT  ma.key_estudiante, ma.periodo, ma.asignatura, AVG(ma2.nivel_nota) promedio_previo_con_docente_actual
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2
    ON ma2.key_estudiante = ma.key_estudiante			-- el mismo estudiante
    AND ma2.key_docente = ma.key_docente				-- el mismo docente
    INNER JOIN f_periodos p2
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- solo filas de interés
    AND p2.orden < p.orden								-- periodos anteriores
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,promedio_previo_con_docente_actual
0,AD60843C610D605B14A18492B1E7E42AA11D3ABD,20202,300MAG006,1.0
1,AD60843C610D605B14A18492B1E7E42AA11D3ABD,20202,300MAG007,1.0
2,973B2D227F2BC391A412ACFDA6DEEF5B14678DDF,20201,300MAG006,-3.0
3,973B2D227F2BC391A412ACFDA6DEEF5B14678DDF,20201,300MAG007,-3.0
4,B827E2B2BB13D1EE7D6E06A017F74662EDA9BD6B,20192,300MAG006,1.0


In [27]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: promedio obtenido si no ha tomado cursos con ese docente antes
DATOS.to_pickle('checkPoints/DATOS26.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 1959)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,cursos_per_canc_antes_otros_dpto_MEA,cursos_per_canc_antes_otros_dpto_PSC,cursos_per_canc_antes_otros_dpto_PSG,cursos_per_canc_antes_otros_dpto_PSV,cursos_per_canc_antes_otros_dpto_TEG,cursos_per_canc_antes_otros_dpto_TMS,docente_experencia_curso,docente_experencia_otros_cursos,cursos_tomados_antes_con_docente,promedio_previo_con_docente_actual
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0


### **27. LIMPIEZA DE DATOS -> DATOS_LIMPIOS**


obtenemos una copia de DATOS para elimnar las columnas que no interesan que será DATOS_LIMPIOS.

Un primer paso es hacer hot_enconding de la columna asignatura, que es la que nos indica de qué asignatura estamos haciendo la predicción.

Al hacer hot_enconding, dos columnas aparecen:  asignatura_300MAG006 y asignatura_300MAG007 de tipo boolean.   ESto permitirá hacer un experimento creando modelos por separado para cada asignatura versus crear un único modelo, con variable de entrada el valor de verdad de ambas asignaturas (una de ellas usando un XOR) y generar una predicción.

In [3]:
# sacamos una copia
DATOS = pd.read_pickle('checkPoints/DATOS26.pkl')
DATOS_LIMPIOS = DATOS.copy()
DATOS_LIMPIOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,id_docente,docente_genero,docente_edad,estado,...,cursos_per_canc_antes_otros_dpto_MEA,cursos_per_canc_antes_otros_dpto_PSC,cursos_per_canc_antes_otros_dpto_PSG,cursos_per_canc_antes_otros_dpto_PSV,cursos_per_canc_antes_otros_dpto_TEG,cursos_per_canc_antes_otros_dpto_TMS,docente_experencia_curso,docente_experencia_otros_cursos,cursos_tomados_antes_con_docente,promedio_previo_con_docente_actual
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,-1,1,43,E,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,-1,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0


In [4]:
DATOS.shape

(7145, 1959)

In [5]:


# # Obtener la lista de nombres de columna y sus tipos
# columnas_y_tipos = [(columna, tipo) for columna, tipo in zip(DATOS_LIMPIOS.columns, DATOS_LIMPIOS.dtypes)]

# # Mostrar la lista de nombres de columna y tipos
# for columna, tipo in columnas_y_tipos:
#     print(f'Columna: {columna}, Tipo: {tipo}')

# hacemos hot_enconding de la columna asignatura
DATOS_LIMPIOS = pd.get_dummies(DATOS_LIMPIOS, columns=['asignatura'])

# eliminamos las columnas que no nos interesan
DATOS_LIMPIOS.drop(['key_estudiante','periodo','colegio','estado','nota','grupo'], axis=1, inplace=True)

# genero nos quedó object y lo convertimos a int
DATOS_LIMPIOS['estudiante_genero'] = DATOS_LIMPIOS['estudiante_genero'].astype(int)
DATOS_LIMPIOS['docente_genero'] = DATOS_LIMPIOS['docente_genero'].astype(int)
DATOS_LIMPIOS['estudiante_programa'] = DATOS_LIMPIOS['estudiante_programa'].astype(int)

# y salvamos el nuevo dataframe tambien.
DATOS_LIMPIOS.to_pickle('checkPoints/DATOS_LIMPIOS.pkl')

In [6]:
DATOS_LIMPIOS.shape

(7145, 1954)

In [11]:
DATOS_LIMPIOS = pd.read_pickle('checkPoints/DATOS26.pkl')
subcadena = 'asignatura'
if (DATOS_LIMPIOS.columns.str.contains(subcadena)).any():
    print(f"Al menos una columna contiene '{subcadena}' como subcadena en el DataFrame DATOS_LIMPIOS.")
else:
    print(f"Ninguna columna contiene '{subcadena}' como subcadena en el DataFrame DATOS_LIMPIOS.")

Al menos una columna contiene 'asignatura' como subcadena en el DataFrame DATOS_LIMPIOS.
